# A joint model CAR (model 1) 
## Two independent processes
 $$ [P , S ] = [P ] [ S ] $$

In [1]:
## Import code:
setwd('/apps/external_plugins/biospytial_rwrapper/biospytial.rwrapper/R/')

In [2]:
rm(list=ls())                                                                                     
source("init_data.R")                                                                             
# load the building function                                                                      
source("joint.binomial.bymCARModel1.R")
## Fix the burnin and sample to be consistent with the other models
n.sample = 10000                                                                                  
burnin=10000                                                                                      
postburnin = burnin +1000                                                                         
thin = 1                                                                                          
verbose = TRUE                           

results  <- joint.binomial.bymCARModel1(formula_S = formula_sample, 
                                        formula_P = formula_sample,
                                        n.sample=n.sample,
                                        data = DataFrame,
                                        burnin=burnin,
                                        postburnin=postburnin,
                                        thin=thin,
                                        verbose=TRUE) 




Loading required package: MASS
Loading required package: Rcpp

Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Setting up the model.
Performing burnin period for Sample Effort Model 
  |======================================================================| 100%
Summarising results.
Finished in  88.9 seconds.
Setting up the model.
Performing burnin period for Presence model 
  |======================================================================| 100%
Summarising results.
Finished in  88.2 seconds.
Sampling from the joint model 10000 post burnin and thinned (if requested) samples.
  |======================================================================| 100%
[1] "Compiling summary for S process"
[1] "Compiling summary for the P process"


In [3]:
results$S$summary.results

,Median,2.5%,97.5%,n.sample,% accept,n.effective,Geweke.diag
(Intercept),10.5346,7.0995,12.0097,10000,53.7,2.3,-31.0
Disttoroadm,-0.0002,-0.0004,-0.0001,10000,53.7,1083.0,-1.9
Populationm,0.0000,-0.0001,0.0001,10000,53.7,1565.5,-0.3
tau2,6.1312,3.7996,9.4912,10000,100.0,15.4,-0.5
sigma2,0.2387,0.0547,0.6632,10000,100.0,4.6,4.3


In [4]:
results$P$summary.results

,Median,2.5%,97.5%,n.sample,% accept,n.effective,Geweke.diag
(Intercept),6.8712,5.4379,8.2628,10000,54.3,6.7,0.9
Disttoroadm,-0.0002,-0.0003,-0.0001,10000,54.3,1260.8,-1.1
Populationm,0.0000,-0.0001,0.0001,10000,54.3,1347.8,0.1
tau2,5.3452,2.3637,8.1601,10000,100.0,8.5,1.2
sigma2,0.0242,0.0113,0.0682,10000,100.0,5.6,2.7


In [5]:
results$summary.results

DIC           p.d          WAIC           p.w          LMPL 
    4278.3202      900.1057     4351.8504      795.1900    -2231.4482 
loglikelihood 
   -1239.0543

In [6]:
fitted_samples = cbind(results$S$samples$fitted,results$P$samples$fitted)

In [7]:
post.sample_p = data.frame(results$S$samples$fitted)
post.presence_p = data.frame(results$P$samples$fitted)
post.joint = data.frame(results$samples$fitted.joint)

In [32]:
## Compute independent joint distribution
library(boot)



fitted_presences = apply(results$P$samples$fitted,MARGIN=1,identity)
fitted_sample = apply(results$S$samples$fitted,MARGIN=1,identity)
fitted_total = apply(results$samples$fitted,MARGIN=1,identity)

re_presence = apply(results$P$samples$psi,MARGIN=1,identity)
re_sample = apply(results$S$samples$psi,MARGIN=1,identity)

In [33]:
getUpperLowerCI <- function(fitted_sample,prefix="",MARGIN=1){
    fp = partial(quantile,probs=c(0.025,0.5,0.975))
    quants =apply(X = fitted_sample,FUN = fp,MARGIN)
    mfitt = apply(fitted_sample,mean,MARGIN = MARGIN)
    vfitt = apply(fitted_sample,var,MARGIN = MARGIN)
    new_data = cbind(t(quants),mfitt,vfitt)
    l = as.list(c("q025","median","q0975","mean","variance"))
    nnames = lapply(l,function (x){return(paste(prefix,x,sep = '_'))})
    colnames(new_data) = nnames
    return(new_data)
}

P = getUpperLowerCI(fitted_presences,prefix = 'P')
S = getUpperLowerCI(fitted_sample,prefix = 'S')
PS = getUpperLowerCI(fitted_total,prefix = 'PS')
Gp = getUpperLowerCI(re_presence,prefix = 'Gp')
Gs = getUpperLowerCI(re_sample,prefix = 'Gs')



In [36]:
## Sample from the posterior
sample.pres <- data.frame(t(fitted_presences)) %>% mutate_all(function(p) rbernoulli(1,p))
sample.samp <- data.frame(t(fitted_sample)) %>% mutate_all(function(p) rbernoulli(1,p))
sample.tot <- data.frame(t(fitted_total)) %>% mutate_all(function(p) rbernoulli(1,p))

In [ ]:
medir dimension
y ver que pasa con colsums


In [ ]:
dataout = cbind(TDF$cellids,P,S,Gp,Gs,PX,PY,PXY)

In [8]:
pss  <- post.sample_p %>% mutate_all(function(p) rbernoulli(1,p))
pps <- post.presence_p %>% mutate_all(function(p) rbernoulli(1,p))
ptot <- post.joint %>% mutate_all(function(p) rbernoulli(1,p))

In [10]:
sumpt <- colSums(ptot)
sumpss <- colSums(pss)
sumpps <- colSums(pps)

In [34]:
nsamples = dim(ptot)[1]

In [35]:
nsamples

[1] 10000

In [14]:
joint_prob_sample <- sumpt / nsamples

In [15]:
prob_pps <- sumpps / nsamples

In [16]:
prob_pss <- sumpss / nsamples

In [17]:
joint_prob_sample_2 = prob_pps * prob_pss

In [18]:
meanfitted = results$fitted.values

In [19]:
## save to posterity
# Save output sample to display in python
## Save the complete chain trace for analysing in Python,
#TDF = read.csv("/outputs/training_data_sample_puebla_p9_abies_pinophyta.csv")
## Order it according to the id of the cell
## This is important because the adjancy matrix rows need to be the same

## No needed because DataFrame object already loaded and without no-neighbour (i.e. same size as fitted values)
#TDF = TDF[order(TDF$cell_ids),]
TDF = DataFrame[order(DataFrame$cellids),]

In [20]:
chain = cbind(TDF$cellids,joint_prob_sample,meanfitted,prob_pps,prob_pss)
file_ = '/outputs/presence_only_models/modelCAR_model1_fitted2.csv'
write.csv(chain,file_)